In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

def etf_rechner_optimiert(
    aktuelles_alter,
    renteneintrittsalter,
    rendite_prozent,
    monatliche_einzahlung,
    erhöhung_prozent,
    entnahme_monatlich
):
    # Konstanten & Initialisierung
    lebenserwartung = 89
    sparjahre = max(0, renteneintrittsalter - aktuelles_alter)
    entnahmejahre = max(0, lebenserwartung - renteneintrittsalter)
    
    r_monatlich = rendite_prozent / 100 / 12
    dynamik_jahre = erhöhung_prozent / 100
    steuer_satz = 0.18 # Inkl. Teilfreistellung
    
    kapital = []
    investiert = []
    kontostand = 0
    summe_investiert = 0
    
    # 1. Sparphase
    for monat in range(1, int(sparjahre * 12) + 1):
        # Dynamik greift nach jedem vollen Jahr (12 Monate)
        jahre_vergangen = (monat - 1) // 12
        aktuelle_rate = monatliche_einzahlung * ((1 + dynamik_jahre) ** jahre_vergangen)
        
        kontostand = kontostand * (1 + r_monatlich) + aktuelle_rate
        summe_investiert += aktuelle_rate
        
        kapital.append(kontostand)
        investiert.append(summe_investiert)

    # 2. Entnahmephase
    n_entnahme_monate = int(entnahmejahre * 12)
    kapital_ende_sparphase = kontostand
    invest_ende_sparphase = summe_investiert
    
    for monat in range(1, n_entnahme_monate + 1):
        if kontostand <= 0:
            kapital.append(0)
            investiert.append(invest_ende_sparphase)
            continue
            
        # Steuerberechnung (Näherung via Gewinnanteil)
        anteil_gewinn = max(0, 1 - (invest_ende_sparphase / kontostand))
        steuer = entnahme_monatlich * anteil_gewinn * steuer_satz
        
        kontostand = kontostand * (1 + r_monatlich) - entnahme_monatlich - steuer
        kapital.append(max(0, kontostand))
        investiert.append(invest_ende_sparphase)

    # Datenaufbereitung für Plot
    gesamt_monate = len(kapital)
    zeitachse = np.linspace(aktuelles_alter, aktuelles_alter + gesamt_monate/12, gesamt_monate)
    kapital = np.array(kapital)
    investiert = np.array(investiert)
    zinsanteil_prozent = ((kapital_ende_sparphase - invest_ende_sparphase) / kapital_ende_sparphase * 100) if kapital_ende_sparphase > 0 else 0

    # Plotting
    plt.figure(figsize=(12, 6))
    
    # Flächen füllen für maximale visuelle Wirkung
    plt.fill_between(zeitachse, investiert, color='#2ecc71', alpha=0.3, label="Eingezahltes Kapital")
    plt.fill_between(zeitachse, investiert, kapital, where=(kapital > investiert), color='#3498db', alpha=0.3, label="Zinseszins-Anteil")
    
    plt.plot(zeitachse, kapital, color='#2980b9', linewidth=3, label="Gesamtvermögen")
    plt.plot(zeitachse, investiert, color='#27ae60', linestyle='--', linewidth=2)
    
    # Vertikale Linien
    plt.axvline(renteneintrittsalter, color='#c0392b', linestyle=':', label="Rentenbeginn")
    
    # Styling & Beschriftung
    plt.title(f"Aufklärungs-Tool: Die Macht des Zinseszinses\n"
              f"Ihr Zinsanteil am Sparziel: {zinsanteil_prozent:.1f}% | "
              f"Endvermögen Sparphase: {kapital_ende_sparphase:,.0f} €", fontsize=14)
    plt.ylabel("Kapitalwert (€)")
    plt.xlabel("Alter (Jahre)")
    plt.grid(True, linestyle=':', alpha=0.6)
    plt.legend(loc='upper left')
    
    # Währung an Y-Achse formatieren
    plt.gca().get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',').replace(',', '.') + ' €'))
    
    plt.tight_layout()
    plt.show()

# Widget-Konfiguration (Zentral gesteuert)
ui_style = {'description_width': '220px'}
ui_layout = {'width': '550px'}

interact(
    etf_rechner_optimiert,
    aktuelles_alter=widgets.IntSlider(value=28, min=18, max=70, step=1, description="Aktuelles Alter", style=ui_style, layout=ui_layout),
    renteneintrittsalter=widgets.IntSlider(value=67, min=25, max=75, step=1, description="Rentenbeginn", style=ui_style, layout=ui_layout),
    rendite_prozent=widgets.FloatSlider(value=7.0, min=0, max=15, step=0.1, description="Erwartete Rendite p.a. (%)", style=ui_style, layout=ui_layout),
    monatliche_einzahlung=widgets.IntSlider(value=200, min=0, max=5000, step=50, description="Start-Sparrate (€)", style=ui_style, layout=ui_layout),
    erhöhung_prozent=widgets.FloatSlider(value=2.0, min=0.0, max=10.0, step=0.1, description="Jährliche Dynamik (%)", style=ui_style, layout=ui_layout),
    entnahme_monatlich=widgets.IntSlider(value=2500, min=0, max=20000, step=100, description="Wunsch-Entnahme (€)", style=ui_style, layout=ui_layout)
);